In [ ]:
# !pip install -r requirements.txt
# !pip install cloudscraper

In [7]:
import os
import csv
import json
import time
import re
from datetime import datetime
from dotenv import load_dotenv

# HTTP 요청 (cloudflare 우회)
import cloudscraper
from bs4 import BeautifulSoup

# SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# ==========================================
# 환경 변수 로드 및 DB Client
# ==========================================
load_dotenv()

class RDSClient:
    def __init__(self):
        db_user = os.getenv('DB_USER')
        db_password = os.getenv('DB_PASSWORD')
        db_host = os.getenv('DB_HOST')
        db_port = os.getenv('DB_PORT', 3306)
        db_name = os.getenv('DB_NAME')

        self.db_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}?charset=utf8mb4"

        try:
            self.engine = create_engine(
                self.db_url,
                pool_size=5,
                max_overflow=10,
                pool_recycle=3600,
                echo=False
            )
            print("✅ DB Engine 생성 완료")
        except Exception as e:
            print(f"❌ Engine 생성 오류: {e}")
            self.engine = None

    def execute(self, query, params=None):
        if not self.engine:
            print("DB 엔진 없음")
            return None

        try:
            with self.engine.connect() as connection:
                stmt = text(query)
                result = connection.execute(stmt, params or {})
                if result.returns_rows:
                    return [dict(row._mapping) for row in result]
                else:
                    connection.commit()
                    return result.rowcount
        except SQLAlchemyError as e:
            # 에러 타입 상세 출력
            error_msg = str(e.orig) if hasattr(e, 'orig') else str(e)
            print(f"⚠️ SQL 오류: {error_msg}")
            return {"error": error_msg}  # None 대신 에러 정보 반환

rds = RDSClient()

# ==========================================
# product_bottom 컬럼 자동 로딩
# ==========================================
cols_result = rds.execute("SHOW COLUMNS FROM product_bottom")
if not cols_result:
    print("❌ 테이블 컬럼 정보를 가져올 수 없습니다.")
    exit(1)

db_cols = [row['Field'] for row in cols_result]
print(f"DB 컬럼 ({len(db_cols)}개):", db_cols)

# INSERT 문 생성
placeholders = ", ".join([f":{c}" for c in db_cols])
insert_sql = f"""
INSERT INTO product_bottom ({", ".join(db_cols)})
VALUES ({placeholders})
"""

# ==========================================
# CSV 로딩
# ==========================================
goods_ids = []
try:
    with open("musinsa_bottom_ids.csv", newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            if row and row[0].strip():
                goods_ids.append(row[0].strip())
except FileNotFoundError:
    print("❌ musinsa_bottom_ids.csv 파일을 찾을 수 없습니다.")
    exit(1)

print(f"총 상품 수: {len(goods_ids)}개")

# ==========================================
# 이미 저장된 상품 불러오기 → 중복 방지
# ==========================================
existing_rows = rds.execute("SELECT product_id FROM product_bottom")
existing_ids = {str(row['product_id']) for row in existing_rows} if existing_rows else set()

print(f"이미 저장된 상품: {len(existing_ids)}개")

# CSV 순서대로 어디까지 저장됐는지 확인
saved_indices = []
not_saved_indices = []
for idx, gid in enumerate(goods_ids):
    if gid in existing_ids:
        saved_indices.append(idx + 1)
    else:
        not_saved_indices.append(idx + 1)

print(f"\n📊 CSV 기준 저장 현황:")
print(f"   저장 완료된 인덱스: {len(saved_indices)}개")
if saved_indices:
    print(f"   - 범위: {min(saved_indices)}번 ~ {max(saved_indices)}번")
    print(f"   - 처음 10개: {saved_indices[:10]}")
    
print(f"\n   저장 필요한 인덱스: {len(not_saved_indices)}개")
if not_saved_indices:
    print(f"   - 범위: {min(not_saved_indices)}번 ~ {max(not_saved_indices)}번")
    print(f"   - 처음 10개: {not_saved_indices[:10]}")
    print(f"   - 마지막 10개: {not_saved_indices[-10:]}")

# ==========================================
# 진행상황 로그 파일 생성
# ==========================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
progress_log = f"progress_log_{timestamp}.txt"

with open(progress_log, "w", encoding="utf-8") as f:
    f.write(f"크롤링 시작 시간: {datetime.now()}\n")
    f.write(f"총 상품 수: {len(goods_ids)}\n")
    f.write(f"이미 저장됨: {len(existing_ids)}\n")
    f.write(f"크롤링 대상: {len(not_saved_indices)}\n\n")

# ==========================================
# cloudscraper 설정
# ==========================================
scraper = cloudscraper.create_scraper()

def clean(name):
    """브랜드명/상품명에서 불필요한 부분 제거"""
    return re.sub(r"-.*", "", name).strip() if name else None

def extract_int(text):
    """텍스트에서 숫자만 추출하여 정수로 반환"""
    if not text:
        return None
    num_str = re.sub(r"[^0-9]", "", text)
    return int(num_str) if num_str else None

def extract_number(text):
    """'만' 단위 포함 숫자 추출 (문자열 반환)"""
    if not text:
        return None
    if "만" in text:
        try:
            num = float(re.sub(r"[^0-9\.]", "", text.replace("만", "")))
            return str(int(num * 10000))
        except:
            return None
    num_str = re.sub(r"[^0-9]", "", text)
    return num_str if num_str else None

# ==========================================
# 크롤링 시작
# ==========================================
print("\n====== 크롤링 시작 ======\n")

success_count = 0
skip_count = 0
error_count = 0

for idx, gid in enumerate(goods_ids):
    csv_index = idx + 1  # CSV 기준 인덱스 (1부터 시작)
    
    # 이미 DB에 있으면 스킵
    if gid in existing_ids:
        skip_count += 1
        continue

    print(f"[CSV {csv_index}번/{len(goods_ids)}] 상품 ID: {gid} 처리 중...")

    try:
        url = f"https://www.musinsa.com/products/{gid}"
        html = scraper.get(url, timeout=10).text
        soup = BeautifulSoup(html, "html.parser")

        # 1. 상품명
        name_tag = soup.select_one("div[class*='GoodsName__Wrap'] span")
        product_name = clean(name_tag.get_text(strip=True)) if name_tag else None
        
        # 상품명이 없으면 크롤링 실패로 간주하고 스킵
        if not product_name:
            print(f"⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품")
            error_count += 1
            with open("error_log.txt", "a", encoding="utf-8") as log:
                log.write(f"CSV {csv_index}번, ID: {gid}, 상품명 크롤링 실패\n")
            with open(progress_log, "a", encoding="utf-8") as f:
                f.write(f"❌ CSV {csv_index}번 | ID: {gid} | 상품명 크롤링 실패\n")
            time.sleep(0.5)
            continue

        # 2. 브랜드
        brand_tag = soup.select_one("div[class*='Brand__Wrap'] span[class*='BrandName']")
        brand = re.sub(r"[0-9\.]+만", "", brand_tag.get_text(strip=True)).strip() if brand_tag else None

        # 3~5 가격
        normal_price = extract_int(soup.select_one("span.line-through").get_text()) if soup.select_one("span.line-through") else None
        sale_price = extract_int(soup.select_one("span[class*='CalculatedPrice']").get_text()) if soup.select_one("span[class*='CalculatedPrice']") else None
        discount = extract_int(soup.select_one("span[class*='DiscountRate']").get_text()) if soup.select_one("span[class*='DiscountRate']") else None

        # 6 카테고리
        cats = soup.select("div[class*='Category__Wrap'] a")
        upper_category = cats[0].get_text(strip=True) if len(cats) > 0 else None
        lower_category = cats[1].get_text(strip=True) if len(cats) > 1 else None

        # 7 성별
        gender = 0
        layout_boxes = soup.select("dl[class*='Layout__Wrap'] div[class*='Layout__Box']")
        if len(layout_boxes) > 1:
            dd = layout_boxes[1].find("dd")
            if dd:
                gtext = dd.get_text(strip=True)
                gender_map = {"남": 1, "여": 2, "공용": 0, "남녀공용": 0}
                gender = gender_map.get(gtext, 0)

        # 누적 판매
        cumulative = None
        for box in layout_boxes:
            dt = box.find("dt")
            dd = box.find("dd")
            if dt and dd and dt.get_text(strip=True) == "누적판매":
                cumulative = dd.get_text(strip=True)
                break

        # 8 별점
        rating_tag = soup.select_one("div[class*='Review__Wrap'] span")
        rating = float(rating_tag.get_text(strip=True)) if rating_tag else None

        # 9 후기수
        review_cnt_tag = soup.select_one("div[class*='Review__Wrap'] span:nth-of-type(2)")
        review_cnt = None
        if review_cnt_tag:
            review_num_str = extract_number(review_cnt_tag.get_text())
            review_cnt = int(review_num_str) if review_num_str else 0

        # 10 관심수
        like_tag = soup.select_one("div[class*='Like__Container'] span")
        like_cnt = extract_number(like_tag.get_text()) if like_tag else None

        # 11 스타일 태그
        tag_items = soup.select("ul[class*='ProductTags__List'] span")
        styles = ",".join([t.get_text(strip=True) for t in tag_items]) if tag_items else None

        size_json = "[]"
        fit_json = json.dumps({"핏": [], "계절감": []}, ensure_ascii=False)

        # 값 매핑 - DB 컬럼 순서대로 딕셔너리 생성
        param_dict = {}
        for col in db_cols:
            if col == 'product_id':
                param_dict[col] = gid
            elif col == 'product_name':
                param_dict[col] = product_name
            elif col == 'brand':
                param_dict[col] = brand
            elif col == 'normal_price':
                param_dict[col] = normal_price
            elif col == 'sale_price':
                param_dict[col] = sale_price
            elif col == 'upper_category':
                param_dict[col] = upper_category
            elif col == 'lower_category':
                param_dict[col] = lower_category
            elif col == 'gender':
                param_dict[col] = gender
            elif col == 'rating':
                param_dict[col] = rating
            elif col == 'like_cnt':
                param_dict[col] = like_cnt
            elif col == 'review_cnt':
                param_dict[col] = review_cnt
            elif col == 'size_json':
                param_dict[col] = size_json
            elif col == 'discount':
                param_dict[col] = discount
            elif col == 'fit_json':
                param_dict[col] = fit_json
            elif col == 'cumulative':
                param_dict[col] = cumulative
            elif col == 'styles':
                param_dict[col] = styles
            else:
                param_dict[col] = None

        # DB 저장
        result = rds.execute(insert_sql, param_dict)
        
        # 결과 처리
        if isinstance(result, dict) and "error" in result:
            # SQL 에러 발생
            error_msg = result["error"]
            error_count += 1
            
            # 에러 타입 판별
            if "Duplicate entry" in error_msg:
                print(f"⚠️ 중복 오류 (이미 DB에 존재) - 스킵")
                existing_ids.add(gid)  # 중복이므로 existing_ids에 추가
                with open(progress_log, "a", encoding="utf-8") as f:
                    f.write(f"⚠️ CSV {csv_index}번 | ID: {gid} | 중복 (이미 존재)\n")
            elif "cannot be null" in error_msg.lower() or "NULL" in error_msg:
                print(f"❌ NULL 오류: {error_msg}")
                with open("error_log.txt", "a", encoding="utf-8") as log:
                    log.write(f"CSV {csv_index}번, ID: {gid}, NULL 오류: {error_msg}\n")
                with open(progress_log, "a", encoding="utf-8") as f:
                    f.write(f"❌ CSV {csv_index}번 | ID: {gid} | NULL 오류\n")
            else:
                print(f"❌ SQL 오류: {error_msg}")
                with open("error_log.txt", "a", encoding="utf-8") as log:
                    log.write(f"CSV {csv_index}번, ID: {gid}, SQL 오류: {error_msg}\n")
                with open(progress_log, "a", encoding="utf-8") as f:
                    f.write(f"❌ CSV {csv_index}번 | ID: {gid} | SQL 오류: {error_msg}\n")
                    
        elif result is not None and result > 0:
            # 저장 성공
            existing_ids.add(gid)
            success_count += 1
            print(f"✅ 저장 완료 (상품명: {product_name})")
            
            # 진행상황 로그에 기록
            with open(progress_log, "a", encoding="utf-8") as f:
                f.write(f"✅ CSV {csv_index}번 | ID: {gid} | 저장 완료\n")
        else:
            # 예상치 못한 결과
            print(f"⚠️ 알 수 없는 오류")
            error_count += 1
            with open("error_log.txt", "a", encoding="utf-8") as log:
                log.write(f"CSV {csv_index}번, ID: {gid}, 알 수 없는 오류\n")
            with open(progress_log, "a", encoding="utf-8") as f:
                f.write(f"❌ CSV {csv_index}번 | ID: {gid} | 알 수 없는 오류\n")

        # 딜레이 (차단 방지)
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ 오류 발생! {gid} - {e}")
        error_count += 1
        with open("error_log.txt", "a", encoding="utf-8") as log:
            log.write(f"CSV {csv_index}번, ID: {gid}, {str(e)}\n")
        with open(progress_log, "a", encoding="utf-8") as f:
            f.write(f"❌ CSV {csv_index}번 | ID: {gid} | 오류: {str(e)}\n")

print("\n====== 모든 작업 완료! ======")
print(f"성공: {success_count}개 | 스킵: {skip_count}개 | 실패: {error_count}개")
print(f"\n📝 진행상황 로그: {progress_log}")
print(f"📝 에러 로그: error_log.txt")

# 최종 저장 상태 요약
with open(progress_log, "a", encoding="utf-8") as f:
    f.write(f"\n크롤링 종료 시간: {datetime.now()}\n")
    f.write(f"성공: {success_count}개 | 스킵: {skip_count}개 | 실패: {error_count}개\n")

✅ DB Engine 생성 완료
DB 컬럼 (16개): ['product_id', 'product_name', 'brand', 'original_price', 'sale_price', 'upper_category', 'lower_category', 'gender', 'rating', 'wish_count', 'review_count', 'size_info', 'discount_rate', 'fit_season', 'cumulative_sales', 'style']
총 상품 수: 59914개
이미 저장된 상품: 3604개

📊 CSV 기준 저장 현황:
   저장 완료된 인덱스: 3604개
   - 범위: 1번 ~ 3664번
   - 처음 10개: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

   저장 필요한 인덱스: 56310개
   - 범위: 50번 ~ 59914번
   - 처음 10개: [50, 64, 101, 307, 346, 378, 379, 380, 382, 392]
   - 마지막 10개: [59905, 59906, 59907, 59908, 59909, 59910, 59911, 59912, 59913, 59914]

====== 크롤링 시작 ======

[CSV 50번/59914] 상품 ID: 4556447 처리 중...
⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품
[CSV 64번/59914] 상품 ID: 4993411 처리 중...
⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품
[CSV 101번/59914] 상품 ID: 4613450 처리 중...
⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품
[CSV 307번/59914] 상품 ID: 3304654 처리 중...
⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품
[CSV 346번/59914] 상품 ID: 3223947 처리 중...
⚠️ 상품명 크롤링 실패 - 페이지 구조 변경 또는 삭제된 상품
[CSV 378번/

KeyboardInterrupt: 